In [62]:
import pandas as pd
import numpy as np
import datetime

In [63]:
import os
from dotenv import load_dotenv

load_dotenv("secret.env")

api_key = os.getenv("OPENROUTER_AI_API_KEY")

if api_key is None:
    print("Error: OPENROUTER_AI_API_KEY not found in .env file or environment variables.")
else:
    print("OPENROUTER_AI_API_KEY loaded successfully.")

OPENROUTER_AI_API_KEY loaded successfully.


In [64]:
import requests
import json
response = requests.get(
  url="https://openrouter.ai/api/v1/auth/key",
  headers={
    "Authorization": f"Bearer {api_key}"
  }
)
print(json.dumps(response.json(), indent=2))

{
  "data": {
    "label": "sk-or-v1-4eb...7b1",
    "limit": null,
    "usage": 0.12919039,
    "is_provisioning_key": false,
    "limit_remaining": null,
    "is_free_tier": true,
    "rate_limit": {
      "requests": 10,
      "interval": "10s"
    }
  }
}


In [65]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)

In [66]:
prompt = """You are a financial news analyst specializing in stock market impact. Your task is to analyze the provided news article, summarize its core content concisely, determine its sentiment (positive, negative, or neutral), and assess its importance to the specified stock.

Here is the news from stock [STOCK] title and body:
---
[TITLE]
---
[BODY]
---

Please provide your analysis in the following format:

**Sentiment:**
[Positive / Negative / Neutral]

**Summary:**
[Your concise summary of the article, typically 2-3 sentences.]

**Reasoning for Sentiment:**
[Brief explanation (1-2 sentences) of why you categorized the sentiment as such, referencing key points or tone from the article.]

**Importance to Stock [STOCK]:**
[1-5, where 1 is minimal importance and 5 is very high importance.] (Answer only in number 1-5)

**Reasoning for Importance:**
[Brief explanation (1-2 sentences) of why you assigned this importance score, referencing specific details from the article that would impact the stock.]"""

In [67]:
df = pd.read_csv("../../DataCollection/Data/News/AMZN_2025-05-26_13-36-10.csv")
display(df.head())

,ticker,publish_date,publish_time,title,body_text,url
0,AMZN,2025-05-23,14:36,Amazon.com is sued over alleged sale of contam...,By Jonathan Stempel\n\n(Reuters) - Amazon.com ...,https://www.investing.com/news/stock-market-ne...
1,AMZN,2025-05-23,14:14,Oracle to buy $40 billion of Nvidia chips for ...,(Reuters) -Oracle will spend around $40 billio...,https://www.investing.com/news/stock-market-ne...
2,AMZN,2025-05-23,09:19,Amazon not seeing notable impact from tariffs ...,Risk Disclosure: Trading in financial instrume...,https://www.investing.com/news/pro/amazon-not-...
3,AMZN,2025-05-23,09:04,Amazon Com Inc receives Investment Bank Analys...,Risk Disclosure: Trading in financial instrume...,https://www.investing.com/news/pro/da-davidson...
4,AMZN,2025-05-23,05:53,Wall St falls as Trump tariff threats spark ma...,By Chibuike Oguh\n\nNEW YORK (Reuters) -U.S. s...,https://www.investing.com/news/stock-market-ne...


In [68]:
df["title"] = df["title"].astype(str)
df["body_text"] = df["body_text"].astype(str)
df.isnull().sum()

ticker          0
publish_date    0
publish_time    0
title           0
body_text       0
url             0
dtype: int64

In [69]:
from tqdm import tqdm
predicted = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Prompting"): 
    title = row["title"]
    body_text = row["body_text"]
    stock = row["ticker"]
    
    prompt_with_data = prompt.replace("[TITLE]", title).replace("[BODY]", body_text).replace("[STOCK]", stock)
    
    response = client.chat.completions.create(
        extra_body={
            "provider": {
            "sort": "throughput"
            }
        },
        model="deepseek/deepseek-chat-v3-0324:free",
        messages=[
            {"role": "system", "content": "You are a financial news analyst."},
            {"role": "user", "content": prompt_with_data}
        ]
    )
    
    predicted.append(response.choices[0].message.content)

Prompting:   0%|          | 0/20 [00:00<?, ?it/s]

Prompting:   0%|          | 0/20 [00:06<?, ?it/s]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1748304000000'}, 'provider_name': None}}, 'user_id': 'user_2xcd4AUf5nAJkkjOkK3zs5b3ejg'}

In [ ]:
predicted = np.array(predicted)
df["predicted"] = predicted

In [ ]:
predicted[8]

"** Stock Analysis: AMZN\n\n**Sentiment:**\n[Neutral]\n\n**Summary:**\nThe article reports on a high-profile charity event in Cannes supporting HIV/AIDS research, attended by numerous celebrities, with concerns raised about U.S. funding cuts to global AIDS programs. The event, organized by amfAR, has raised significant funds for research since 1993.\n\n**Reasoning for Sentiment:**\nThe article maintains a neutral tone, primarily reporting facts about the event and celebrity attendance, while mentioning concerns about funding cuts without taking a strong positive or negative stance.\n\n**Importance to stock AMZN:**\n[1 - Minimal importance]\n\n**Reasoning for importance:**\nThe article's connection to Amazon (AMZN) is extremely tangential, mentioning only that Jeff Bezos' fiancée attended the event. This has no material impact on Amazon's business operations or financial performance."

In [ ]:
print([x.split('\n') for x in predicted])

[['**Sentiment:**  ', 'Negative  ', '', '**Summary:**  ', 'Amazon is facing a class-action lawsuit over allegations of selling rice products contaminated with heavy metals, including arsenic, cadmium, lead, and mercury. The lawsuit claims these products, sold under well-known brands like Ben’s Original and Whole Foods’ 365, pose health risks, particularly to children, without proper warnings.  ', '', '**Reasoning for Sentiment:**  ', 'The article highlights serious allegations of health risks associated with Amazon’s products, which could damage the company’s reputation and lead to financial penalties. The tone is critical, emphasizing the lack of warnings and potential harm to consumers.  ', '', '**Importance to stock AMZN:**  ', '3  ', '', '**Reasoning for Importance:**  ', 'While the lawsuit could lead to financial and reputational damage, Amazon’s diversified business model mitigates the impact. However, repeated legal issues or regulatory scrutiny could affect consumer trust and s

In [ ]:
print(df['title'][8],predicted[8])

Celebrities unite at exclusive Cannes gala to support AIDS research after Trump cuts By Reuters ** Stock Analysis: AMZN

**Sentiment:**
[Neutral]

**Summary:**
The article reports on a high-profile charity event in Cannes supporting HIV/AIDS research, attended by numerous celebrities, with concerns raised about U.S. funding cuts to global AIDS programs. The event, organized by amfAR, has raised significant funds for research since 1993.

**Reasoning for Sentiment:**
The article maintains a neutral tone, primarily reporting facts about the event and celebrity attendance, while mentioning concerns about funding cuts without taking a strong positive or negative stance.

**Importance to stock AMZN:**
[1 - Minimal importance]

**Reasoning for importance:**
The article's connection to Amazon (AMZN) is extremely tangential, mentioning only that Jeff Bezos' fiancée attended the event. This has no material impact on Amazon's business operations or financial performance.


In [ ]:
list= df["predicted"].apply(lambda x: x.split("\n"))

In [ ]:
df["sentiment"] = list.apply(lambda x: x[1] if len(x) > 1 else None)
df["importance"] = list.apply(lambda x: x[10] if len(x) > 10 else None)
df["summary"] = list.apply(lambda x: x[4] if len(x) > 4 else None)

In [ ]:
df[["title", "sentiment", "importance", "summary"]]

,title,sentiment,importance,summary
0,Amazon.com is sued over alleged sale of contam...,Negative,3,Amazon is facing a class-action lawsuit over a...
1,Oracle to buy $40 billion of Nvidia chips for ...,Positive,3,Oracle will invest $40 billion in Nvidia's hig...
2,Amazon not seeing notable impact from tariffs ...,Neutral,2,The article states that Amazon has not yet exp...
3,Amazon Com Inc receives Investment Bank Analys...,Neutral,1,The article is a standard disclaimer about the...
4,Wall St falls as Trump tariff threats spark ma...,,None,
5,Workday Inc receives Investment Bank Analyst R...,Neutral,1 (Minimal Importance),The article is a standard risk disclosure and ...
6,Nike receives Investment Bank Analyst Rating U...,Neutral,1 (Minimal Importance),The article is a standard risk disclosure and ...
7,This CEO ’is making all the right moves to unl...,Positive,2,The article highlights that Amazon's CEO is ta...
8,Celebrities unite at exclusive Cannes gala to ...,,,
9,Disney seeks to stop YouTube from hiring veter...,Neutral,2 (Low importance),Disney is suing YouTube (Alphabet) to block th...
